# Segmenting and Clustering Neighborhoods in Toronto

#### I will explore, segment, and cluster the neighborhoods in the city of Toronto.

In order to retrieve the information about neighborhoods in Toronto at the Wikipedia, it will be used the Python package called Beatiful Soup.

## *Assignment - Part 1*

In [ ]:
#install BeautifulSoup package (run if necessary)
!pip install beautifulsoup4

In [1]:
#import packages
import pandas as pd
from bs4 import BeautifulSoup
import requests # library to handle requests

Get wikipage as html and create a BeautifulSoup Object

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url)
#print (source.status_code)
#soup = BeautifulSoup(source.text,"html.parser")
soup = BeautifulSoup(source.text,"lxml")
#print(soup.prettify())

Creation of a list with the data

In [3]:
list_with_data = []
table = soup.find('table')

for tr in table.findAll('tr'):
    aux_list = []
    for td in tr.findAll('td'):
        data = td.text.strip()
        aux_list.append(data)
    if len(aux_list)==3:
        list_with_data.append(aux_list)

### **3.1 - The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood**

In [4]:
#The dataframe will have 3 Columns = PostalCode, Borough, and Neighborhood
columns_name = ['PostalCode', 'Borough','Neighborhood']
df = pd.DataFrame(data = list_with_data, columns=columns_name)
df.head(5)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### **3.2 - Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.**

In [5]:
index_names = df[df['Borough'] == 'Not assigned'].index

df.drop(index_names, inplace = True)
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### **3.3 - More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.**

In the current wikipedia version, each Postal Code has already several neighborhoods. However we can check if there is any postal code duplicated

In [6]:
df_duplicated = df[df.duplicated(['PostalCode'])]
df_duplicated

,PostalCode,Borough,Neighborhood


### **3.4 - If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.**

In [7]:
#First chech which Borough doesn't have a neighborhood assigned
df_neighborhood_check = df[df.Neighborhood == 'Not assigned']
df_neighborhood_check.shape[0]

0

The dataframe doesn't have any 'Not Assigned' neighborhood in the Borough.

### **3.5 - Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.**

In [9]:
#Need to reset index column
df = df.reset_index(drop = True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### **3.6 - In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.**

Let's check how many rows has the dataframe

In [10]:
print('The dataframe has {} rows'.format(df.shape[0]))

The dataframe has 103 rows
